In [12]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [13]:
def scrape_courses(program_name, URL):
    driver = webdriver.Chrome()
    driver.get(URL)
    elem = driver.find_elements("xpath","//*[contains(text(),'" + program_name + "')]")

    for ele in elem:
        ele.click()
        time.sleep(10)
        
        soup = BeautifulSoup(driver.page_source,'html.parser')
        tables = soup.find_all("tbody")
        del tables[-1]
        
        course_lists = []
        course_number = []
        class_number = []
        class_status = []

        topic_names = ['Topic: Managing Big Data', 'Topic: Text Data Mining and Analytics', 
                       'Topic: Data Analytics and Visualization', 'Topic: Global Field Intensive: Entrepreneurship and Innov']

        for table in tables:
            class_number.append(table.find_all("div")[1].find_all("div")[0].text)
            course = table.find("b").text
            if course in topic_names:
                course_number.append(table.find("p").find("b").text)
            else:
                course_number.append(course)

            status = table.find_all("div")[1].find_all("div")[3].text
            if status == 'Requires Department Consent\n':
                class_status.append(table.find_all("div")[1].find_all("div")[4].text)
            else:
                class_status.append(status)

        for i in range(len(tables)):
            course_lists.append([course_number[i], class_number[i], class_status[i]])

    return course_lists


In [14]:
program_name = "Management & Systems MA/GC (MASY1-GC)"
URL = "https://sis.nyu.edu/psc/csprod/EMPLOYEE/SA/c/NYU_SR.NYU_CLS_SRCH.GBL"

In [15]:
scrape_courses(program_name, URL)

[['MASY1-GC 1200', 'Class#: 3739 ', 'Class Status: Open '],
 ['MASY1-GC 1200', 'Class#: 3963 ', 'Class Status: Closed '],
 ['MASY1-GC 1200', 'Class#: 4053 ', 'Class Status: Open '],
 ['MASY1-GC 1200', 'Class#: 4013 ', 'Class Status: Closed '],
 ['MASY1-GC 1200', 'Class#: 4079 ', 'Class Status: Closed '],
 ['MASY1-GC 1200', 'Class#: 3976 ', 'Class Status: Open '],
 ['MASY1-GC 1200', 'Class#: 19832 ', 'Class Status: Open '],
 ['MASY1-GC 1200', 'Class#: 23813 ', 'Class Status: Closed '],
 ['MASY1-GC 1200', 'Class#: 3988 ', 'Class Status: Closed '],
 ['MASY1-GC 1200', 'Class#: 4052 ', 'Class Status: Closed '],
 ['MASY1-GC 1200', 'Class#: 4449 ', 'Class Status: Closed '],
 ['MASY1-GC 1210', 'Class#: 4551 ', 'Class Status: Open '],
 ['MASY1-GC 1210', 'Class#: 3959 ', 'Class Status: Open '],
 ['MASY1-GC 1210', 'Class#: 4054 ', 'Class Status: Closed '],
 ['MASY1-GC 1210', 'Class#: 4081 ', 'Class Status: Closed '],
 ['MASY1-GC 1210', 'Class#: 3977 ', 'Class Status: Open '],
 ['MASY1-GC 1210', '

In [16]:
course_lists = scrape_courses(program_name, URL)
print(len(course_lists))

186


In [21]:
course_df = pd.DataFrame(scrape_courses(program_name, URL), columns=['course_number','class_number','class_status'])

In [22]:
course_df.head()

,course_number,class_number,class_status
0,MASY1-GC 1200,Class#: 3739,Class Status: Open
1,MASY1-GC 1200,Class#: 3963,Class Status: Closed
2,MASY1-GC 1200,Class#: 4053,Class Status: Open
3,MASY1-GC 1200,Class#: 4013,Class Status: Closed
4,MASY1-GC 1200,Class#: 4079,Class Status: Closed


In [ ]:
# Using pd.DataFrame

import smtplib
import time
from email.message import EmailMessage

while True:
    course_df = pd.DataFrame(scrape_courses(program_name, URL), columns=['course_number','class_number','class_status'])

    course_wanted = 'MASY1-GC 1200'
    class_wanted = 'Class#: 3739 '
    status_wanted = 'Class Status: Open '

    condition = (course_df["course_number"] == course_wanted) & (course_df["class_number"] == class_wanted) & (course_df["class_status"] == status_wanted)
    courses_condition = course_df[condition]

    if not courses_condition.empty:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls() 
        server.login('from@gmail.com', 'password')
        message = f"The course {course_wanted} with {class_wanted} is now open."
        msg = EmailMessage()
        msg.set_content(message)
        msg['Subject'] = 'Course Open Notification'
        msg['From'] = 'from@gmail.com'
        msg['To'] = 'to@gmail.com'
        server.send_message(msg)
        server.quit()
        
    time.sleep(4)


In [ ]:
# Using list of lists

import smtplib
from email.message import EmailMessage

while True:
    course_lists = scrape_courses(program_name, URL)
    
    for course in course_lists:
        course_wanted = 'MASY1-GC 1200'
        class_wanted = 'Class#: 3739 '
        if course[0] == course_wanted and course[1] == class_wanted and course[2] == 'Class Status: Open ':
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls() 
            server.login('from@gmail.com', 'password')
            message = f"The course {course_wanted} with {class_wanted} is now open."
            msg = EmailMessage()
            msg.set_content(message)
            msg['Subject'] = 'Course Open Notification'
            msg['From'] = 'from@gmail.com'
            msg['To'] = 'to@gmail.com'
            server.send_message(msg)
            server.quit()
            
    time.sleep(4)